In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
directory = 'testdata'
list_raw_files = []

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        path = (directory + '/' + filename)
        list_raw_files.append(path)
list_raw_files

['testdata/mealData5.csv', 'testdata/Nomeal5.csv']

In [3]:
list_of_df = []
import csv
for i in list_raw_files:
    df = pd.read_csv(i)
    df.drop(df.columns[[0]], axis=1, inplace=True)
    list_of_df.append(df)
list_of_df

[        0      1      2      3      4      5      6           7           8  \
 0   125.0  127.0  128.0  128.0  131.0  135.0  139.0  141.000000  144.000000   
 1   211.0  216.0  220.0  221.0  219.0  218.0  213.0  209.000000  207.000000   
 2   182.0  194.0  204.0  218.0  239.0  246.0  246.0  242.000000  242.000000   
 3   179.0  170.0  163.0  163.0  167.0  175.0  184.0  190.000000  193.000000   
 4   240.0  238.0  236.0  236.0  230.0  227.0  226.0  229.000000  221.000000   
 5   182.0  180.0  170.0  154.0  149.0  153.0  159.0  170.000000  172.000000   
 6   192.0  203.0  212.0  214.0  214.0  214.0  215.0  217.000000  217.000000   
 7   141.0  141.0  142.0  142.0  145.0  150.0  155.0  161.000000  166.000000   
 8   185.0  193.0  200.0  206.0  207.0  203.0  200.0  204.000000  204.000000   
 9   296.0  294.0  294.0  292.0  290.0  294.0  295.0  293.000000  290.000000   
 10  173.0  184.0  185.0  189.0  192.0  191.0  190.0  192.000000  194.000000   
 11  197.0  200.0  204.0  207.0  211.0  

In [4]:
def calculate_iqr(row):
    median = row.quantile(0.5)
    quantile1 = row.quantile(0.25)
    quantile3 = row.quantile(0.75)
    iqr = quantile3 - quantile1
    return iqr

In [5]:
def calculate_ZC(row):
    zero_x = 0

    for idx, item in enumerate(row[:-1]):
        if row[idx] == row[idx+1]:
            zero_x +=1
        
    return zero_x

In [6]:
def calculate_GFI(row):
    sum_squares = 0
    for idx, item in enumerate(row[:-1]):
        currdiff = row[idx] - row[idx+1]
        sum_squares += currdiff ** 2
    return np.sqrt(sum_squares/(len(row)-1))

In [7]:
for currdf in list_of_df:
    currdf['mean'] = currdf.mean(axis=1)
    currdf['lbgv'] = currdf.min(axis=1)
    currdf['hbgv'] = currdf.max(axis=1)
    currdf['median'] = currdf.median(axis=1)
    currdf['iqr'] = currdf.apply(calculate_iqr, axis=1)
    currdf['std'] = currdf.std(axis=1)
    currdf['MAG'] = currdf.diff(axis=1).abs().sum(axis=1)/2.5
    currdf['ZC'] = currdf.apply(calculate_ZC, axis=1)
    currdf['CV'] = currdf.std(axis=1)/currdf.mean(axis=1)
    currdf['GFI'] = currdf.apply(calculate_GFI, axis=1)
    currdf['GCF'] = currdf.apply(calculate_GFI, axis=1)/currdf.mean(axis=1)
    currdf['MAGE'] = currdf.max(axis=1) - currdf.min(axis=1)

In [8]:
for idx,currdf in enumerate(list_of_df):
    if idx <= 0:
        currdf['Label'] = 1
    else:
        currdf['Label'] = 0

In [9]:
feat_df_list = []
for currdf in list_of_df:
    newdf = currdf[['mean','lbgv','hbgv','median','iqr','std','MAG','CV','ZC','GFI','GCF','MAGE']]
    feat_df_list.append(newdf)
feat_df_list

[          mean   lbgv   hbgv      median         iqr         std         MAG  \
 0   147.633333  125.0  169.0  148.500000   12.000000   26.019905  128.407962   
 1   163.966667  107.0  221.0  158.000000   72.250000   41.790183  218.857260   
 2   211.800000  163.0  254.0  211.000000   59.500000   40.516635  229.233346   
 3   213.000000  163.0  256.0  215.000000   57.500000   40.928648  257.628541   
 4   188.666667  141.0  240.0  187.000000   65.750000   40.537359  199.518390   
 5   131.000000   93.0  182.0  127.000000   47.500000   31.573757  194.570497   
 6   188.066667  142.0  228.0  201.000000   62.750000   38.073031  196.024121   
 7   194.533333  141.0  262.0  200.000000   56.000000   44.966294  237.613483   
 8   246.666667  185.0  301.0  252.000000   76.750000   47.774358  250.890257   
 9   241.150000  165.0  296.0  250.000000  102.000000   54.165140  328.453944   
 10  201.633333  173.0  238.0  200.000000   19.000000   34.207228  190.082891   
 11  218.733333  192.0  262.

In [10]:
df_pca = pd.concat(feat_df_list, ignore_index=True)
df_pca

,mean,lbgv,hbgv,median,iqr,std,MAG,CV,ZC,GFI,GCF,MAGE
0,147.633333,125.0,169.0,148.500000,12.000000,26.019905,128.407962,0.281504,4,35.629345,0.272681,168.727319
1,163.966667,107.0,221.0,158.000000,72.250000,41.790183,218.857260,0.336874,1,53.774735,0.361030,220.663126
2,211.800000,163.0,254.0,211.000000,59.500000,40.516635,229.233346,0.290146,4,57.269244,0.301990,253.709854
3,213.000000,163.0,256.0,215.000000,57.500000,40.928648,257.628541,0.293427,2,65.023188,0.339491,257.335114
4,188.666667,141.0,240.0,187.000000,65.750000,40.537359,199.518390,0.305287,2,50.562750,0.298464,239.701536
5,131.000000,93.0,182.0,127.000000,47.500000,31.573757,194.570497,0.332635,0,47.536862,0.397352,194.570497
6,188.066667,142.0,228.0,201.000000,62.750000,38.073031,196.024121,0.293950,8,49.603239,0.293648,227.706352
7,194.533333,141.0,262.0,200.000000,56.000000,44.966294,237.613483,0.317268,3,60.647587,0.344735,261.682732
8,246.666667,185.0,301.0,252.000000,76.750000,47.774358,250.890257,0.292710,1,64.635771,0.292899,300.707290
9,241.150000,165.0,296.0,250.000000,102.000000,54.165140,328.453944,0.317132,2,81.198875,0.370955,328.136812


In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
x = df_pca.values
x = StandardScaler().fit_transform(x)
col_mean = np.nanmean(x, axis=0)
inds = np.where(np.isnan(x))
x[inds] = np.take(col_mean, inds[1])

In [12]:
import pickle
pca_filename = 'finalized_pca.sav'
pca = pickle.load(open(pca_filename, 'rb'))

In [13]:
new_prinComps = pca.transform(x)

In [14]:
testprincipalDf = pd.DataFrame(data = new_prinComps, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4','principal component 5'])
#testprincipalDf = pd.DataFrame(data = new_prinComps, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4','principal component 5','principal component 6','principal component 7','principal component 8','principal component 9'])

testprincipalDf

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5
0,-3.814437,0.120800,0.064969,-0.125038,0.239580
1,-1.088908,1.543994,-0.367937,-0.659977,0.011530
2,-0.758491,-0.462099,0.437021,-0.326666,0.211919
3,-0.269696,-0.196012,-0.213881,-0.055824,0.278147
4,-1.191127,0.175749,-0.078253,-0.827870,0.081458
5,-2.242678,2.101347,-0.958893,-0.123412,-0.032845
6,-1.651505,-0.121838,1.631729,-0.343151,0.295697
7,-0.386079,0.372762,0.153393,-0.087715,0.064360
8,0.673257,-1.041661,-0.251588,-0.775830,0.169574
9,1.973676,0.102002,0.152584,-0.357589,0.357108


In [15]:
df_init = pd.concat(list_of_df, ignore_index=True)
df_init

,0,1,2,3,4,5,6,7,8,9,...,median,iqr,std,MAG,ZC,CV,GFI,GCF,MAGE,Label
0,125.0,127.0,128.0,128.0,131.0,135.0,139.0,141.000000,144.000000,145.000000,...,148.500000,12.000000,26.019905,128.407962,4,0.281504,35.629345,0.272681,168.727319,1
1,211.0,216.0,220.0,221.0,219.0,218.0,213.0,209.000000,207.000000,188.000000,...,158.000000,72.250000,41.790183,218.857260,1,0.336874,53.774735,0.361030,220.663126,1
2,182.0,194.0,204.0,218.0,239.0,246.0,246.0,242.000000,242.000000,246.000000,...,211.000000,59.500000,40.516635,229.233346,4,0.290146,57.269244,0.301990,253.709854,1
3,179.0,170.0,163.0,163.0,167.0,175.0,184.0,190.000000,193.000000,195.000000,...,215.000000,57.500000,40.928648,257.628541,2,0.293427,65.023188,0.339491,257.335114,1
4,240.0,238.0,236.0,236.0,230.0,227.0,226.0,229.000000,221.000000,212.000000,...,187.000000,65.750000,40.537359,199.518390,2,0.305287,50.562750,0.298464,239.701536,1
5,182.0,180.0,170.0,154.0,149.0,153.0,159.0,170.000000,172.000000,159.000000,...,127.000000,47.500000,31.573757,194.570497,0,0.332635,47.536862,0.397352,194.570497,1
6,192.0,203.0,212.0,214.0,214.0,214.0,215.0,217.000000,217.000000,217.000000,...,201.000000,62.750000,38.073031,196.024121,8,0.293950,49.603239,0.293648,227.706352,1
7,141.0,141.0,142.0,142.0,145.0,150.0,155.0,161.000000,166.000000,171.000000,...,200.000000,56.000000,44.966294,237.613483,3,0.317268,60.647587,0.344735,261.682732,1
8,185.0,193.0,200.0,206.0,207.0,203.0,200.0,204.000000,204.000000,213.000000,...,252.000000,76.750000,47.774358,250.890257,1,0.292710,64.635771,0.292899,300.707290,1
9,296.0,294.0,294.0,292.0,290.0,294.0,295.0,293.000000,290.000000,286.000000,...,250.000000,102.000000,54.165140,328.453944,2,0.317132,81.198875,0.370955,328.136812,1


In [16]:
model_filename = 'finalized_model.sav'
classifier = pickle.load(open(model_filename, 'rb'))

In [17]:
pr=classifier.predict(testprincipalDf)
from sklearn.metrics import accuracy_score
score = accuracy_score(df_init['Label'], pr)
print (pr)
df_init['Label']

[1 0 0 0 0 0 0 0 1 0 0 0 1 1 1 1 1 0 0 0 1 1 1 0 0 0 0 1 0 0 1 1 0 1 1 1 1
 0 0 0 1 1 1 0 0 0 1 1 0 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 0 0 1 1 0 1 0 1
 1 0 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1]


0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
      ..
71     0
72     0
73     0
74     0
75     0
76     0
77     0
78     0
79     0
80     0
81     0
82     0
83     0
84     0
85     0
86     0
87     0
88     0
89     0
90     0
91     0
92     0
93     0
94     0
95     0
96     0
97     0
98     0
99     0
100    0
Name: Label, Length: 101, dtype: int64

In [18]:
score

0.48514851485148514

In [19]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(df_init['Label'],pr))
print(classification_report(df_init['Label'],pr))

[[25 25]
 [27 24]]
              precision    recall  f1-score   support

           0       0.48      0.50      0.49        50
           1       0.49      0.47      0.48        51

    accuracy                           0.49       101
   macro avg       0.49      0.49      0.49       101
weighted avg       0.49      0.49      0.49       101

